edwith의 부스트 코스 : "파이토치로 시작하는 딥러닝 기초" 를 바탕으로 작성되었습니다.  
https://www.boostcourse.org/ai214

# Weight initialization

- Why good initialization?
- RBM / DBN
- Xavier / He initialization
- Full code : mnist_nn_xavier
- Full code : mnist_nn_deep

## Why good initialization?
Need to set the initial weight values wisely  
* Not all 0's (neural network 를 업데이트할 때, backpropagation 알고리즘을 사용하는데 gradient를 계산을 해서 weight를 업데이트하는데 weight가 0이게 되면 모든 gradient값이 0으로 바뀌게 됨. - 학습이 불가능)
* Challenging issue
* Hinton et al. (2006) " A Fast Learning Algorithm for Deep Belief Nets" - Restricted Boltzmann Machine (RBM)을 이용해서 weight initialization을 하게 되면 성능이 올라간다는 논문

실제로 initialization 을 무작위로 설정하는 것 보다 weight initialization 방식을 적용한 모델들이 더 성능이 좋았다는 연구결과가 존재   
    $\therefore$ weight initialization 을 잘 하는 것이 딥러닝의 성능에 큰 영향을 미치는 것을 확인할 수 있음. . 

## RBM / DBN
* Restricted Boltzmann Machine
    * Restricted : no connections within a layer
    * KL divergence : compare actual to recreation
    
*How we can use RBM to initialize weights?*
 * Apply the RBM idea on adjacent two layers as a pre-training step
 * Continue the first process to all layers
 * This will set weights 
 * Example : Deep Belief Network
     * Weight initialized by RBM

## Xavier / He initialization
* No need to use complicated RBM for weight initializations 
* Simple methods are OK
    * **Xavier initialization** : X. Glorot and Y.Bengio, "Understanding the difficulty of training deep feedforward neural networks." in International conference on artificial intelligence and statistics, 2010
    * **He initialization** : K.He, X.Zang, S.Ren, and J.Sun, "Delving Deep into Rectifiers: Suprpassing Human-Level Performance on ImangeNet Classification", 2015

무작위적으로 weights를 initialized 하는 것이 아니라, layer에 특성에 따라서 다르게 initialization 을 해야한다.

### Xavier
RBM 처럼 pre-training step 을 거치거나 하는 복잡한 형태가 아닌 수식으로써 간단하게 weights 를 초기화 하는 방법 2가지가 존재함. 
* Xavier Normal initialization
$$W \sim N(0, Var(W))$$
$$Var(W) = \sqrt{\dfrac{2}{n_{in}+n_{out}}}$$

n_in 은 레이어의 input 수를 의미하고, out은 레이어의 output의 수를 의미함. 

* Xavier Uniform initialization
$$W \sim U(-\sqrt{\dfrac{6}{n_{in}+n_{out}}}, + \sqrt{\dfrac{6}{n_{in}+n_{out}}})$$

### He initialization
Xavier initialization 의 변형, n_out term 없이 사용되는 초기화 방법

* He Normal initialization
$$W \sim N(0, Var(W))$$
$$Var(W) = \sqrt{\dfrac{2}{n_{in}}}$$

n_in 은 레이어의 input 수를 의미하고, out은 레이어의 output의 수를 의미함. 

* He Uniform initialization
$$W \sim U(-\sqrt{\dfrac{6}{n_{in}}}, + \sqrt{\dfrac{6}{n_{in}}})$$


## Full code : mnist_nn_xavier

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda': 
    torch.cuda.manual_seed_all(777)

In [3]:
learning_rate = 0.001
traing_epochs = 15
batch_size = 100

In [4]:
mnist_train = dsets.MNIST(root='../MNIST_data/',
                         train=True,
                         transform=transforms.ToTensor(),
                         download=False)

mnist_test = dsets.MNIST(root='../MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=False)

In [5]:
data_loader = torch.utils.data.DataLoader(mnist_train,
                                         batch_size=batch_size,
                                         shuffle=True,
                                         drop_last=True)

In [7]:
# nn layer
linear1 = torch.nn.Linear(784, 256, bias=True)
linear2 = torch.nn.Linear(256, 256, bias=True)
linear3 = torch.nn.Linear(256, 10, bias=True)
relu = torch.nn.ReLU()

In [8]:
# xavier_uniform initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)

Parameter containing:
tensor([[ 0.0749,  0.1136,  0.0031,  ...,  0.0042,  0.1258,  0.0327],
        [ 0.0348,  0.0790, -0.1096,  ..., -0.0396, -0.0324,  0.1384],
        [ 0.1407,  0.1386, -0.0059,  ...,  0.0106, -0.0942, -0.0712],
        ...,
        [-0.0367,  0.1250,  0.0028,  ...,  0.0386, -0.0834, -0.1472],
        [ 0.0443,  0.0738, -0.1421,  ...,  0.1075, -0.0641,  0.0136],
        [ 0.1313,  0.0921, -0.0932,  ..., -0.0239, -0.0482, -0.1415]],
       requires_grad=True)

In [9]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3).to(device)

In [10]:
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# train
total_batch = len(data_loader)
for epoch in range(traing_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        x_train = X.view(-1, 28 * 28).to(device)
        y_train = Y.to(device)
        
        prediction = model(x_train)
        
        cost = criterion(prediction, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('Epoch : {:4d}/{}, Cost : {:.6f}'.format(epoch, traing_epochs, cost.item()))
    
print('Training finished')

In [ ]:
# test
with torch.no_grad():
    x_test = mnist_test.data.view(-1, 28*28).float().to(device)
    y_test = mnist_test.target.float().to(device)
    
    pred = model(x_test)
    correct = torch.argmax(pred, 1) == y_test
    
    accuracy = correct.float().mean()
    print('Accuracy : {:5f}%'.format(accuracy * 100))
    
    # get random one and predict
    r = random.randint(0, len(mnist_test) - 1)
    
    x_single = mnist_test.data[r].view(-1, 28 * 28).float().to(device)
    y_single = mnist_test.target[r].float().to(device)
    
    pred = model(x_single)
    
    print('Label : ', y_single.item())
    print('Prediction', torch.argmax(pred, 1).item())

## Full code : mnist_nn_deep

In [13]:
# nn layer
linear1 = torch.nn.Linear(784, 512, bias=True)
linear2 = torch.nn.Linear(512, 512, bias=True)
linear3 = torch.nn.Linear(512, 512, bias=True)
linear4 = torch.nn.Linear(512, 512, bias=True)
linear5 = torch.nn.Linear(512, 10, bias=True)
relu = torch.nn.ReLU()

In [14]:
# xavier_uniform initialization
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0714, -0.0682,  0.0077,  ...,  0.0413,  0.0087, -0.0673],
        [-0.0304, -0.0842, -0.0350,  ..., -0.0041, -0.0595,  0.0582],
        [ 0.0194,  0.0091, -0.0018,  ..., -0.0852,  0.0177,  0.0429],
        ...,
        [-0.0685,  0.0552,  0.0257,  ...,  0.0862, -0.1019, -0.0520],
        [ 0.0563,  0.0515,  0.0608,  ...,  0.0279,  0.0008,  0.0822],
        [-0.0808,  0.0378,  0.0452,  ..., -0.0638,  0.0668, -0.0434]],
       requires_grad=True)

In [15]:
model = torch.nn.Sequential(linear1, relu, linear2, relu, linear3, relu, linear4, relu, linear5).to(device)

In [ ]:
total_batch = len(data_loader)
for epoch in range(traing_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        x_train = X.view(-1, 28 * 28).to(device)
        y_train = Y.to(device)
        
        prediction = model(x_train)
        
        cost = criterion(prediction, y_train)
        
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
    
    print('Epoch : {:4d}/{}, Cost : {:.6f}'.format(epoch, traing_epochs, cost.item()))
    
print('Training finished')